In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 468.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 35.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=9516eb5d84335b0863690c5122c73719cb2e52fe857cf371d22b4f85dc4c8f87
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━

In [ ]:
# Импортируем библиотеки
import os
import re
import string
import sentence_transformers
from sentence_transformers import SentenceTransformer

import numpy as np
import pandas as pd
import plotly.express as px

from tqdm import tqdm

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

pd.options.plotting.backend = "plotly"

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import spacy
from spacy.lang.ru import stop_words
from spacy.lang.ru.examples import sentences

nlp = spacy.load("ru_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

Apple рассматривает возможность покупки стартапа из Соединённого Королевства за $1 млрд
Apple PROPN nsubj
рассматривает VERB ROOT
возможность NOUN obj
покупки NOUN nmod
стартапа NOUN nmod
из ADP case
Соединённого ADJ amod
Королевства PROPN nmod
за ADP case
$ NOUN nmod
1 NUM appos
млрд NOUN punct


In [ ]:
!wget wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JC7ABwERBFMTjxM4LJzbryWEkoLwn0hF' -O data.csv

--2024-11-15 06:07:17--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2024-11-15 06:07:17--  https://docs.google.com/uc?export=download&id=1JC7ABwERBFMTjxM4LJzbryWEkoLwn0hF
Resolving docs.google.com (docs.google.com)... 74.125.20.139, 74.125.20.100, 74.125.20.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.20.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1JC7ABwERBFMTjxM4LJzbryWEkoLwn0hF&export=download [following]
--2024-11-15 06:07:17--  https://drive.usercontent.google.com/download?id=1JC7ABwERBFMTjxM4LJzbryWEkoLwn0hF&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.107.132, 2607:f8b0:400e:c0d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.107.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

1  Скачаем данные и разобъем их на обучающую и тестовую

In [ ]:
def prep_target(y, a=1, b=-1):
  return y.apply(lambda x: a if x == '+' else b)


data = pd.read_csv('data.csv')
X = data['отзывы']
y = data['разметка']

X_train, X_test, y_train, y_test = train_test_split(
    X, prep_target(y), test_size=0.3)

X_train.head()

,отзывы
130,"К покупке не рекомендую! Я считаю , что телефо..."
106,"Купил неделю назад в магазине, аккумулятора хв..."
54,"Шустрый, без косяков, все как заявлено продавцом"
146,не тратьте деньги. более тупящего телефона я н...
7,С тех пор как перешёл с аналога на Rapid Quant...


2  Exploratory data analysis


In [ ]:
# Проверка на наличие nan значений.
X_train.isna().sum()

0

In [ ]:
# Гистограмма длины твитов
X_train.apply(lambda x: len(x)).hist(
    labels={"value": "tweets length"}, title='Tweets hist')

In [ ]:
# Корреляция в зависимости от длины текста
pd.DataFrame([X_train.apply(lambda x: len(x)), y_train]).T.corr()

,отзывы,разметка
отзывы,1.000000,-0.014147
разметка,-0.014147,1.000000


In [ ]:
# Проверка на то, что нету пустых строк. Длина строки больше 11.
mask = X_train.apply(lambda x: len(x))
X_train[mask == min(mask)]

,отзывы
51,Супер
10,Круто


3 TfidfVectorizer с предварительной лемматизацией

In [ ]:
def prep(message):
    message = message.lower()
    punctuation = string.punctuation

    for i in punctuation:
        message = message.replace(i, '')

    message = message.split()
    message = list(filter(lambda x: x not in punctuation, message))
    message = [ i.lemma_ for i in nlp(' '.join(message))]
    message = list(filter(lambda x: x not in stop_words.STOP_WORDS, message))
    message = ' '.join(message)

    return message


train_corpus = list(map(prep, X_train.tolist()))
test_corpus = list(map(prep, X_test.tolist()))

vectorizer = TfidfVectorizer(max_features=40)
vectorizer.fit(train_corpus)

train_corpus_vectorized = vectorizer.transform(train_corpus).toarray()
test_corpus_vectorized = vectorizer.transform(test_corpus).toarray()

4 Embedding

In [ ]:
# Получаю ембидинги текстов
prec = 'float32'
model = SentenceTransformer("ai-forever/ru-en-RoSBERTa", truncate_dim=40)
train_corpus_embed = model.encode(X_train.tolist(),
                                  normalize_embeddings=True,
                                  precision = prec)
test_corpus_embed = model.encode(X_test.tolist(),
                                         normalize_embeddings=True,
                                         precision = prec)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.99M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

### 5  Обучение модели

5.1 Support vector classifier

In [ ]:
clf1 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf1.fit(train_corpus_embed, y_train)
accuracy_score(clf1.predict(test_corpus_embed), y_test)

0.9047619047619048

5.2 Regresion classifier

In [ ]:
clf2 = make_pipeline(StandardScaler(), RidgeClassifier())
clf2.fit(train_corpus_embed, y_train)
accuracy_score(clf2.predict(test_corpus_embed), y_test)

0.8888888888888888

5.3 Regresion classifierCV


In [ ]:
clf3 = make_pipeline(StandardScaler(), RidgeClassifierCV(cv=4))
clf3.fit(train_corpus_embed, y_train)
accuracy_score(clf3.predict(test_corpus_embed), y_test)

0.8888888888888888

5.4 KNNClasifier

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier

clf4 = KNeighborsClassifier(metric='cosine')
clf4.fit(train_corpus_embed, y_train)
accuracy_score(clf3.predict(test_corpus_embed), y_test)

0.8888888888888888

6 Q classifier

In [ ]:
def var_layer(x):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)

    qml.U1(x[0], wires=0)
    qml.U1(x[1], wires=1)

    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)

    qml.CNOT(wires=[0, 1])
    qml.U1(np.pi * np.cos(x[0]) * np.cos(x[1]), wires=1)
    qml.CNOT(wires=[0, 1])

In [ ]:
import pennylane as qml

In [ ]:
dev = qml.device("default.qubit", 2)

In [ ]:
@qml.qnode(dev)
def dot_prod(x1, x2):
    var_layer(x1)
    qml.adjoint(var_layer)(x2)

    return qml.probs(wires=[0, 1])

In [ ]:
def q_dot_prod(x, i, j):
    x1 = (x[i, 0], x[i, 1])
    x2 = (x[j, 0], x[j, 1])

    return dot_prod(x1, x2)[0]

In [ ]:
print(qml.draw(dot_prod)(train_corpus_embed, y_train))

KeyError: 1

In [ ]:
def make_mat(X):
      gram_mat = np.zeros((X.shape[0], X.shape[0]))

      for i in tqdm(range(X.shape[0])):
          for j in range(X.shape[0]):
              if i == j:
                  gram_mat[i, j] = 1
              if i > j:
                  r = q_dot_prod(X, i, j)
                  gram_mat[i, j] = r
                  gram_mat[j, i] = r
      return gram_mat


test_mat = make_mat(np.concatenate((test_corpus_embed, np.zeros((len(train_corpus_embed) - len(test_corpus_embed), len(test_corpus_embed[0]))))))
train_mat = make_mat(train_corpus_embed)

100%|██████████| 147/147 [00:49<00:00,  3.00it/s]


In [ ]:
model = SVC(kernel="precomputed")
model.fit(train_mat, y_train)

SVC(kernel='precomputed')

In [ ]:
a = np.concatenate((test_corpus_embed, np.zeros((len(train_corpus_embed) - len(test_corpus_embed), len(test_corpus_embed[0])))))
b = make_mat(a)

100%|██████████| 147/147 [00:49<00:00,  2.98it/s]


In [ ]:
accuracy_score(model.predict(test_mat)[:len(y_test)], y_test)

AttributeError: 'SentenceTransformer' object has no attribute 'predict'

7 Predict

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('test50.csv')
print(data)
X_new = data['Отзывы']
y_new = data['разметка']

                                               Отзывы разметка
0   Очень разочарован QPhone. Ожидал, что квантовы...        -
1   Очень быстро разряжается. Просто полное разоча...        -
2   Ну что тут сказать, телефон хороший, флагман т...        +
3   QPhone Pro MAX просто бомба! Быстрее любого см...        +
4   Слишком дорого для массового рынка – не каждый...        -
5   Многие приложения несовместимы с квантовой арх...        -
6                 Точно буду советовать своим друзьям        +
7   За три месяца так и не смог привыкнуть к интер...        -
8   С каждым обновлением кажется, что работает всё...        -
9   Не берите Qphone pro Max. Только деньги потратите        -
10  Скорость – просто огонь, но если сравнивать с ...        +
11  Телефон кипит нагревается когда работает на ру...        -
12  После QPhone Pro MAX обычные смартфоны кажутся...        +
13  Наверное для фотографирования котиков он не оч...        +
14  На вид очень стильный, но отпечатки на экране ...  

In [ ]:
def prep(message):
    message = message.lower()
    punctuation = string.punctuation

    for i in punctuation:
        message = message.replace(i, '')

    message = message.split()
    message = list(filter(lambda x: x not in punctuation, message))
    message = [ i.lemma_ for i in nlp(' '.join(message))]
    message = list(filter(lambda x: x not in stop_words.STOP_WORDS, message))
    message = ' '.join(message)

    return message


corpus = list(map(prep, X_new.tolist()))


vectorizer = TfidfVectorizer(max_features=40)
vectorizer.fit(corpus)

corpus_vectorized = vectorizer.transform(corpus).toarray()


In [ ]:
# Эмбединг
def prep_target_(y, a=1, b=-1):
  return y.apply(lambda x: a if x == '+' else b)

prec = 'float32'
model = SentenceTransformer("ai-forever/ru-en-RoSBERTa", truncate_dim=40)
corpus_embed = model.encode(X_new.tolist(),
                                  normalize_embeddings=True,
                                  precision = prec)

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Делаем предсказание
clf1.predict(corpus_embed)

array([ 1, -1,  1,  1, -1, -1, -1, -1,  1,  1,  1, -1,  1,  1, -1,  1, -1,
        1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1, -1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1, -1])